<a href="https://colab.research.google.com/github/saptarshidatta96/Vgg16-Classifier/blob/main/model_training_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import os.path
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
import os 
import xml.etree.ElementTree as ET
from collections import OrderedDict
import PIL
from PIL import Image
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pwd

/content


In [ ]:
!tar -xvf "/content/gdrive/MyDrive/VOCtrainval_11-May-2012.tar" -C "/content/"  

Streaming output truncated to the last 5000 lines.
VOCdevkit/VOC2012/SegmentationClass/2008_001874.png
VOCdevkit/VOC2012/SegmentationClass/2008_001876.png
VOCdevkit/VOC2012/SegmentationClass/2008_001882.png
VOCdevkit/VOC2012/SegmentationClass/2008_001885.png
VOCdevkit/VOC2012/SegmentationClass/2008_001895.png
VOCdevkit/VOC2012/SegmentationClass/2008_001896.png
VOCdevkit/VOC2012/SegmentationClass/2008_001926.png
VOCdevkit/VOC2012/SegmentationClass/2008_001966.png
VOCdevkit/VOC2012/SegmentationClass/2008_001971.png
VOCdevkit/VOC2012/SegmentationClass/2008_001992.png
VOCdevkit/VOC2012/SegmentationClass/2008_001997.png
VOCdevkit/VOC2012/SegmentationClass/2008_002032.png
VOCdevkit/VOC2012/SegmentationClass/2008_002043.png
VOCdevkit/VOC2012/SegmentationClass/2008_002064.png
VOCdevkit/VOC2012/SegmentationClass/2008_002066.png
VOCdevkit/VOC2012/SegmentationClass/2008_002067.png
VOCdevkit/VOC2012/SegmentationClass/2008_002073.png
VOCdevkit/VOC2012/SegmentationClass/2008_002079.png
VOCdevkit/VOC

In [ ]:
def parseXML(tree):
  roww = OrderedDict()
  for elements in tree.iter():
    if elements.tag == 'filename':
      roww['{}'.format(elements.tag)] = str(elements.text)
    if elements.tag =='object':
      for element in elements:
        if element.tag == 'name':
          roww['{}'.format(element.tag)] = str(element.text)
  return roww



In [ ]:
df_annot = []
for file in os.listdir('/content/VOCdevkit/VOC2012/Annotations'):
  if file.startswith('.') == 0:
    tree = ET.parse(os.path.join('/content/VOCdevkit/VOC2012/Annotations', file))
    roww = parseXML(tree)
    df_annot.append(roww)

  df_annotation = pd.DataFrame(df_annot)

In [ ]:
df_annotation

,filename,name
0,2012_001183.jpg,person
1,2009_004118.jpg,person
2,2008_004384.jpg,bottle
3,2011_005414.jpg,person
4,2009_004438.jpg,person
...,...,...
17120,2009_000239.jpg,person
17121,2010_002353.jpg,sheep
17122,2010_005619.jpg,person
17123,2011_002447.jpg,sheep


In [ ]:
list_of_unique_class =df_annotation['name'].unique()
list_of_files = df_annotation['filename'].unique()
annotations_list = df_annotation.values.tolist()

In [ ]:
list_of_unique_class

array(['person', 'bottle', 'aeroplane', 'bird', 'boat', 'dog',
       'pottedplant', 'cat', 'sofa', 'bicycle', 'chair', 'car', 'train',
       'cow', 'horse', 'tvmonitor', 'diningtable', 'bus', 'sheep',
       'motorbike'], dtype=object)

In [ ]:
annotations_list

[['2012_001183.jpg', 'person'],
 ['2009_004118.jpg', 'person'],
 ['2008_004384.jpg', 'bottle'],
 ['2011_005414.jpg', 'person'],
 ['2009_004438.jpg', 'person'],
 ['2008_007357.jpg', 'aeroplane'],
 ['2012_000940.jpg', 'person'],
 ['2010_006167.jpg', 'person'],
 ['2007_006477.jpg', 'person'],
 ['2009_003285.jpg', 'bird'],
 ['2007_005358.jpg', 'boat'],
 ['2008_000472.jpg', 'bird'],
 ['2009_000410.jpg', 'dog'],
 ['2008_004353.jpg', 'person'],
 ['2011_002200.jpg', 'person'],
 ['2012_004119.jpg', 'person'],
 ['2009_003966.jpg', 'pottedplant'],
 ['2012_004189.jpg', 'person'],
 ['2008_003045.jpg', 'cat'],
 ['2010_006749.jpg', 'person'],
 ['2008_003271.jpg', 'sofa'],
 ['2011_001447.jpg', 'person'],
 ['2011_005850.jpg', 'person'],
 ['2011_005253.jpg', 'person'],
 ['2012_000934.jpg', 'person'],
 ['2009_000975.jpg', 'bicycle'],
 ['2010_001261.jpg', 'chair'],
 ['2008_004711.jpg', 'car'],
 ['2011_002202.jpg', 'person'],
 ['2007_004241.jpg', 'boat'],
 ['2010_006664.jpg', 'person'],
 ['2008_008583.jpg'

In [ ]:
os.chdir('/content/VOCdevkit/VOC2012/JPEGImages')
if os.path.isdir('train/person') is False:
  for class_name in list_of_unique_class:
    os.makedirs('train/{}'.format(class_name))
    os.makedirs('validation/{}'.format(class_name))

  for index, i in enumerate(annotations_list):
    if index %4 !=0:
      shutil.move(i[0], 'train/{}'.format(i[1]))
    if index %4 == 0:
      shutil.move(i[0], 'validation/{}'.format(i[1]))

  os.chdir('/../..')

In [ ]:
train_path = '/content/VOCdevkit/VOC2012/JPEGImages/train'
validation_path = '/content/VOCdevkit/VOC2012/JPEGImages/validation'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
.flow_from_directory(train_path, target_size= (224,224), batch_size = 128, class_mode = 'categorical')

Found 12843 images belonging to 20 classes.


In [ ]:
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
.flow_from_directory(validation_path, target_size= (224,224), batch_size = 128, class_mode = 'categorical')

Found 4282 images belonging to 20 classes.


In [ ]:

vgg16_model = tf.keras.applications.vgg16.VGG16()

2021-08-29 16:54:21.933161: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-29 16:54:22.471183: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-29 16:54:22.472055: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-29 16:54:22.473933: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-29 16:54:22.474796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

553476096/553467096 [==============================] - 6s 0us/step


In [ ]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)


for layer in model.layers:
  layer.trainable = False

model.add(Dense(20, activation = 'softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches,
          steps_per_epoch=len(train_batches),
          validation_data=valid_batches,
          validation_steps=len(valid_batches),
          epochs=15,
          verbose=2)

2021-08-29 16:54:35.909109: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15


2021-08-29 16:54:39.475616: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8004
2021-08-29 16:55:02.858454: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.62GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


101/101 - 233s - loss: 1.2855 - accuracy: 0.6804 - val_loss: 1.0979 - val_accuracy: 0.6950
Epoch 2/15
101/101 - 136s - loss: 0.7806 - accuracy: 0.7667 - val_loss: 1.0251 - val_accuracy: 0.7102
Epoch 3/15
101/101 - 136s - loss: 0.6001 - accuracy: 0.8065 - val_loss: 1.0575 - val_accuracy: 0.7268
Epoch 4/15
101/101 - 135s - loss: 0.4892 - accuracy: 0.8452 - val_loss: 1.0644 - val_accuracy: 0.7057
Epoch 5/15
101/101 - 136s - loss: 0.3889 - accuracy: 0.8722 - val_loss: 1.1046 - val_accuracy: 0.7139
Epoch 6/15
101/101 - 136s - loss: 0.3187 - accuracy: 0.8976 - val_loss: 1.1368 - val_accuracy: 0.7172
Epoch 7/15
101/101 - 136s - loss: 0.2619 - accuracy: 0.9200 - val_loss: 1.1481 - val_accuracy: 0.7184
Epoch 8/15
101/101 - 137s - loss: 0.2232 - accuracy: 0.9333 - val_loss: 1.1576 - val_accuracy: 0.7123
Epoch 9/15
101/101 - 136s - loss: 0.1926 - accuracy: 0.9455 - val_loss: 1.2263 - val_accuracy: 0.7212
Epoch 10/15
101/101 - 136s - loss: 0.1652 - accuracy: 0.9579 - val_loss: 1.2059 - val_accurac

In [ ]:
!pwd

/


In [ ]:
os.chdir('/content/')

In [ ]:
model.save('trained_model.h5')

In [ ]:
model.save('/content/gdrive/MyDrive/models/',save_format= 'tf')

2021-08-29 17:30:19.594073: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2021-08-29 17:30:21.424367: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 411041792 exceeds 10% of free system memory.
2021-08-29 17:30:21.712666: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 67108864 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/models/assets


2021-08-29 17:30:24.653 Assets written to: /content/gdrive/MyDrive/models/assets
